In [2]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
import scipy.sparse as sp
from scipy.special import expit
import pickle
import csv
import scipy.sparse as sparse
import copy
import itertools
from lightfm import LightFM
import lightfm.evaluation
import sys
import implicit
plt.style.use('ggplot')

In [3]:
raw_df = pd.read_csv('cat_de_3year.csv', encoding = "utf-8")#, nrows = 100000)

In [4]:
## to start let us take a sample of our data
sample_df = raw_df.sample(10000000)
sample_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID
856462,207293269,49383481,0,04-FEB-15,950427,15532,8790000,1.0000,6103,4975
15501439,283452474,66978650,0,06-APR-16,1780177,88098445,13559228,1.0000,66374,47955
16695032,286732143,67752586,0,24-APR-16,1028167,41349368,2549110,1.0000,52417,39756
25651234,340759068,80105386,0,02-JAN-17,910041,40904116,15783502,1.0000,52309,39845
7044977,240387845,57061592,0,26-AUG-15,1478680,68599300,8333663,0.7678,62612,45360


In [5]:
# remove non-used columns
#sample_df.drop(['SCM_N_ORDER_POS_NR','SCM_N_ORDER_NR_NC', 'SCM_N_ORDER_NR_NC', 'PPR_N_KEY_PHYS_PRODUCT', 'SCM_N_ORDER_NR', 'ART_N_KEY_ARTICLE','PPR_N_PHY_PRODUCT_ID'], axis=1, inplace=True)
sample_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID
856462,207293269,49383481,0,04-FEB-15,950427,15532,8790000,1.0000,6103,4975
15501439,283452474,66978650,0,06-APR-16,1780177,88098445,13559228,1.0000,66374,47955
16695032,286732143,67752586,0,24-APR-16,1028167,41349368,2549110,1.0000,52417,39756
25651234,340759068,80105386,0,02-JAN-17,910041,40904116,15783502,1.0000,52309,39845
7044977,240387845,57061592,0,26-AUG-15,1478680,68599300,8333663,0.7678,62612,45360


In [6]:
sample_renamed_df = sample_df.rename(index=str, columns = {'CUS_N_KEY_CUSTOMER': 'uid', 
                                                   'ART_N_KEY_ARTICLE': 'mid', 
                                                   'TDT_T_KEY_ORDER_DATE': 'date', # it looks like this works a bit better
                                                   'SCM_D_QUANTITY': 'eventStrength'})
sample_renamed_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,date,mid,PPR_N_KEY_PHYS_PRODUCT,uid,eventStrength,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID
856462,207293269,49383481,0,04-FEB-15,950427,15532,8790000,1.0000,6103,4975
15501439,283452474,66978650,0,06-APR-16,1780177,88098445,13559228,1.0000,66374,47955
16695032,286732143,67752586,0,24-APR-16,1028167,41349368,2549110,1.0000,52417,39756
25651234,340759068,80105386,0,02-JAN-17,910041,40904116,15783502,1.0000,52309,39845
7044977,240387845,57061592,0,26-AUG-15,1478680,68599300,8333663,0.7678,62612,45360


In [7]:
n_users = sample_renamed_df.uid.unique().shape[0]
n_items = sample_renamed_df.mid.unique().shape[0]

print('Number of users: {}'.format(n_users))
print('Number of models: {}'.format(n_items))
print('Sparsity: {:4.3f}%'.format(float(sample_renamed_df.shape[0]) / float(n_users*n_items) * 100))

Number of users: 904997
Number of models: 20814
Sparsity: 0.053%


In [8]:
sample_renamed_df.uid[2]

2549110

In [43]:
raw_df[raw_df['CUS_N_KEY_CUSTOMER'] == 15783502].head(30)

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID
25387607,338243366,79528354,79528354,19-DEC-16,1916791,32772670,15783502,0.6035,49725,37908
25387608,338243366,79528354,79528354,19-DEC-16,1916791,11235,15783502,2.3965,11078,9538
25387609,338243364,79528354,79528354,19-DEC-16,1545998,12085,15783502,0.3026,26186,19451
25387695,338243364,79528354,79528354,19-DEC-16,1545998,12090,15783502,0.1831,26184,19449
25387696,338243364,79528354,79528354,19-DEC-16,1545998,53365386,15783502,0.0237,56512,42948
25387697,338243364,79528354,79528354,19-DEC-16,1545998,27437865,15783502,0.3059,45318,34303
25387900,338243364,79528354,79528354,19-DEC-16,1545998,69310175,15783502,0.1848,62710,15952
25387901,338243365,79528354,79528354,19-DEC-16,793527,33467073,15783502,1.0000,49701,38047
25651099,340759072,80105386,0,02-JAN-17,1040411,49419052,15783502,1.0000,55406,42240
25651100,340759066,80105386,0,02-JAN-17,1056581,49887243,15783502,1.0000,55568,22015


In [10]:
raw_df['ART_N_KEY_ARTICLE'].unique().size

21260

In [11]:
raw_df['PPR_N_PHY_ARTICLE_ID'].unique().size

5095

In [12]:
raw_df['PPR_N_PHY_PRODUCT_ID'].unique().size

2288

In [13]:
raw_df[raw_df['ART_N_KEY_ARTICLE'] == 1281816].head(30)

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID
4843,210626987,50166085,0,25-FEB-15,1281816,53757806,10893687,0.2500,57358,43259
4844,210626987,50166085,0,25-FEB-15,1281816,53757818,10893687,0.2500,57353,43259
4845,210626987,50166085,0,25-FEB-15,1281816,53757804,10893687,0.2501,57359,43259
4846,210626987,50166085,0,25-FEB-15,1281816,53757814,10893687,0.2499,57357,43259
20760,209132321,49817093,49817093,16-FEB-15,1281816,53757804,11116570,0.2501,57359,43259
20798,209132321,49817093,49817093,16-FEB-15,1281816,53757818,11116570,0.2500,57353,43259
20831,209132321,49817093,49817093,16-FEB-15,1281816,53757814,11116570,0.2498,57357,43259
20832,209132321,49817093,49817093,16-FEB-15,1281816,53757806,11116570,0.2500,57358,43259
24274,216758914,51570665,0,03-APR-15,1281816,53757806,10495145,0.2500,57358,43259
24789,216758914,51570665,0,03-APR-15,1281816,53757814,10495145,0.2498,57357,43259


While implicit recommendations excel where data is sparse, it can often be helpful to make the interactions matrix a little more dense. We limited our data collection to models that had at least 5 likes. However, it may not be the case that every user has liked at least 5 models. Let's go ahead and knock out users that liked fewer than 5 models. This could possibly mean that some models end up with fewer than 5 likes once these users are knocked out, so we will have to alternate back and forth knocking users and models out until things stabilize.

In [14]:
def threshold_likes(df, uid_min, mid_min):
    n_users = df.uid.unique().shape[0]
    n_items = df.mid.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Starting likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    
    done = False
    while not done:
        starting_shape = df.shape[0]
        mid_counts = df.groupby('uid').mid.count()
        df = df[~df.uid.isin(mid_counts[mid_counts < mid_min].index.tolist())]
        uid_counts = df.groupby('mid').uid.count()
        df = df[~df.mid.isin(uid_counts[uid_counts < uid_min].index.tolist())]
        ending_shape = df.shape[0]
        if starting_shape == ending_shape:
            done = True
    
    assert(df.groupby('uid').mid.count().min() >= mid_min)
    assert(df.groupby('mid').uid.count().min() >= uid_min)
    
    n_users = df.uid.unique().shape[0]
    n_items = df.mid.unique().shape[0]
    sparsity = float(df.shape[0]) / float(n_users*n_items) * 100
    print('Ending likes info')
    print('Number of users: {}'.format(n_users))
    print('Number of models: {}'.format(n_items))
    print('Sparsity: {:4.3f}%'.format(sparsity))
    return df

In [15]:
df_lim = threshold_likes(sample_renamed_df, 5, 5)

Starting likes info
Number of users: 904997
Number of models: 20814
Sparsity: 0.053%
Ending likes info
Number of users: 391169
Number of models: 19092
Sparsity: 0.121%


Nice, we're above 0.1% which should be suitable for making decent recommendations. We now need to map each uid and mid to a row and column, respectively, for our interactions, or "likes" matrix. This can be done simply with Python dictionaries

In [16]:
# Create mappings
mid_to_idx = {}
idx_to_mid = {}
for (idx, mid) in enumerate(df_lim.mid.unique().tolist()):
    mid_to_idx[mid] = idx
    idx_to_mid[idx] = mid
    
uid_to_idx = {}
idx_to_uid = {}
for (idx, uid) in enumerate(df_lim.uid.unique().tolist()):
    uid_to_idx[uid] = idx
    idx_to_uid[idx] = uid

The last step is to actually build the matrix. We will use sparse matrices so as not to take up too much memory. Sparse matrices are tricky because they come in many forms, and there are giant performance tradeoffs between them. Below is a super slow way to build a likes matrix. I tried running %%timeit but got bored waiting for it to finish.

In [18]:
def map_ids(row, mapper):
    return mapper[row]

In [19]:
%%timeit
I = df_lim.uid.apply(map_ids, args=[uid_to_idx]).as_matrix()
J = df_lim.mid.apply(map_ids, args=[mid_to_idx]).as_matrix()
V = np.ones(I.shape[0])
likes = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
likes = likes.tocsr()

14.7 s ± 812 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
I = df_lim.uid.apply(map_ids, args=[uid_to_idx]).as_matrix()
J = df_lim.mid.apply(map_ids, args=[mid_to_idx]).as_matrix()
V = np.ones(I.shape[0])
likes = sparse.coo_matrix((V, (I, J)), dtype=np.float64)
likes = likes.tocsr()

## Cross-validation: Splitsville

Okay, we got a likes matrix and need to split it into training and test matrices. I do this a bit trickily (which is maybe a word?). I would like to track precision@k as my optimization metric later. A k of 5 would be nice. However, if I move 5 items from training to test for some of the users, then they may not have any data left in the training set (remember they had a minimum 5 likes). Thus, the train_test_split only looks for people who have at least 2*k (10 in this case) likes before moving some of their data to the test set. This obviously biases our cross-validation towards users with more likes. So it goe

In [21]:
def train_test_split(ratings, split_count, fraction=None):
    """
    Split recommendation data into train and test sets
    
    Params
    ------
    ratings : scipy.sparse matrix
        Interactions between users and items.
    split_count : int
        Number of user-item-interactions per user to move
        from training to test set.
    fractions : float
        Fraction of users to split off some of their
        interactions into test set. If None, then all 
        users are considered.
    """
    # Note: likely not the fastest way to do things below.
    train = ratings.copy().tocoo()
    test = sparse.lil_matrix(train.shape)
    
    if fraction:
        try:
            user_index = np.random.choice(
                np.where(np.bincount(train.row) >= split_count * 2)[0], 
                replace=False,
                size=np.int32(np.floor(fraction * train.shape[0]))
            ).tolist()
        except:
            print(('Not enough users with > {} '
                  'interactions for fraction of {}')\
                  .format(2*k, fraction))
            raise
    else:
        user_index = range(train.shape[0])
        
    train = train.tolil()

    for user in user_index:
        test_ratings = np.random.choice(ratings.getrow(user).indices, 
                                        size=split_count, 
                                        replace=False)
        train[user, test_ratings] = 0.
        # These are just 1.0 right now
        test[user, test_ratings] = ratings[user, test_ratings]
   
    
    # Test and training are truly disjoint
    assert(train.multiply(test).nnz == 0)
    return train.tocsr(), test.tocsr(), user_index

In [22]:
train, test, user_index = train_test_split(likes, 5, fraction=0.2)

## Cross-validation: Grid search

Now that the data is split into training and test matrices, let's run a giant grid search to optimize our hyperparameters. We have four parameters that we would like to optimize:
1. num_factors: The number of latent factors, or degree of dimensionality in our model.
2. regularization: Scale of regularization for both user and item factors.
3. alpha: Our confidence scaling term.
4. iterations: Number of iterations to run Alternating Least Squares optimization.

I am going to track mean squared error (MSE) and precision at k (p@k), but I mainly care about the later. I've written some functions below to help with metric calculations and making the training log printout look nice. I am going to calculate a bunch of learning curves (that is, evaluate performance metrics at various stages of the training process) for a bunch of different hyperparameter combinations. Props to scikit-learn for being open source and letting me basically crib their GridSearchCV code

In [23]:
from sklearn.metrics import mean_squared_error
def calculate_mse(model, ratings, user_index=None):
    preds = model.predict_for_customers()
    if user_index:
        return mean_squared_error(ratings[user_index, :].toarray().ravel(),
                                  preds[user_index, :].ravel())
    
    return mean_squared_error(ratings.toarray().ravel(),
                              preds.ravel())

In [24]:
def precision_at_k(model, ratings, k=5, user_index=None):
    if not user_index:
        user_index = range(ratings.shape[0])
    ratings = ratings.tocsr()
    precisions = []
    # Note: line below may become infeasible for large datasets.
    predictions = model.predict_for_customers()
    for user in user_index:
        # In case of large dataset, compute predictions row-by-row like below
        # predictions = np.array([model.predict(row, i) for i in xrange(ratings.shape[1])])
        top_k = np.argsort(-predictions[user, :])[:k]
        labels = ratings.getrow(user).indices
        precision = float(len(set(top_k) & set(labels))) / float(k)
        precisions.append(precision)
    return np.mean(precisions)        

In [25]:
def print_log(row, header=False, spacing=12):
    top = ''
    middle = ''
    bottom = ''
    for r in row:
        top += '+{}'.format('-'*spacing)
        if isinstance(r, str):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, int):
            middle += '| {0:^{1}} '.format(r, spacing-2)
        elif isinstance(r, float):
            middle += '| {0:^{1}.5f} '.format(r, spacing-2)
        bottom += '+{}'.format('='*spacing)
    top += '+'
    middle += '|'
    bottom += '+'
    if header:
        print(top)
        print(middle)
        print(bottom)
    else:
        print(middle)
        print(top)

In [26]:
def learning_curve(model, train, test, epochs, k=5, user_index=None):
    if not user_index:
        user_index = range(train.shape[0])
    prev_epoch = 0
    train_precision = []
    train_mse = []
    test_precision = []
    test_mse = []
    
    headers = ['epochs', 'p@k train', 'p@k test',
               'mse train', 'mse test']
    print_log(headers, header=True)
    
    for epoch in epochs:
        model.iterations = epoch - prev_epoch
        if not hasattr(model, 'user_vectors'):
            model.fit(train)
        else:
            model.fit_partial(train)
        train_mse.append(calculate_mse(model, train, user_index))
        train_precision.append(precision_at_k(model, train, k, user_index))
        test_mse.append(calculate_mse(model, test, user_index))
        test_precision.append(precision_at_k(model, test, k, user_index))
        row = [epoch, train_precision[-1], test_precision[-1],
               train_mse[-1], test_mse[-1]]
        print_log(row)
        prev_epoch = epoch
    return model, train_precision, train_mse, test_precision, test_mse

In [27]:
def grid_search_learning_curve(base_model, train, test, param_grid,
                               user_index=None, patk=5, epochs=range(2, 40, 2)):
    """
    "Inspired" (stolen) from sklearn gridsearch
    https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py
    """
    curves = []
    keys, values = zip(*param_grid.items())
    for v in itertools.product(*values):
        params = dict(zip(keys, v))
        this_model = copy.deepcopy(base_model)
        print_line = []
        for k, v in params.items():
            setattr(this_model, k, v)
            print_line.append((k, v))

        print(' | '.join('{}: {}'.format(k, v) for (k, v) in print_line))
        _, train_patk, train_mse, test_patk, test_mse = learning_curve(this_model, train, test,
                                                                epochs, k=patk, user_index=user_index)
        curves.append({'params': params,
                       'patk': {'train': train_patk, 'test': test_patk},
                       'mse': {'train': train_mse, 'test': test_mse}})
    return curves

Please note that the below parameter grid is fucking huge and took like 2 days to run on my 6-year old 4-core i5. It turns out that the performance metrics functions are actually a good bit slower than the training process. These functions could be simply paralellized - something for me to do on a later date.

In [28]:
# param_grid = {'num_factors': [10, 20, 40, 80, 120],
#              'regularization': [0.0, 1e-5, 1e-3, 1e-1, 1e1, 1e2],
#              'alpha': [1, 10, 50, 100, 500, 1000]}
param_grid = {'num_factors': [10, 20],
              'regularization': [0.0, 1e-5],
              'alpha': [1 ]}

In [29]:
#base_model = implicit.als.AlternatingLeastSquares(factors=10)

In [30]:
#curves = grid_search_learning_curve(base_model, train, test,
#                                    param_grid,
#                                    user_index=user_index,
#                                    patk=5)

# fixing implicit implementation:

In [31]:
base_model = implicit.als.AlternatingLeastSquares()

In [32]:
likes_t = likes.transpose()
likes_t.shape

(19092, 391169)

In [33]:
# train the model on a sparse matrix of item/user/confidence weights
base_model.fit(likes_t)

100%|██████████| 15.0/15 [00:43<00:00,  3.04s/it]


In [34]:
user_items = likes_t.T.tocsr()

In [35]:
user_items.shape

(391169, 19092)

In [44]:
recommendations = base_model.recommend(uid_to_idx[15783502], user_items)

# find related items
#related = base_model.similar_items(mid_to_idx[])

In [45]:
recommendations

[(64, 0.30870855),
 (454, 0.23755044),
 (1071, 0.22287682),
 (611, 0.16433668),
 (25, 0.14973506),
 (781, 0.14618538),
 (5372, 0.13294923),
 (2960, 0.13039686),
 (417, 0.12510076),
 (1592, 0.12273858)]

In [48]:
idx_to_mid[611]

1886303